In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time
tf.enable_eager_execution()

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
## First 250 characters 
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))
  

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [12]:
sequences

<DatasetV1Adapter shapes: (101,), types: tf.int64>

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
dataset

<DatasetV1Adapter shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [15]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [17]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
  
  tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
  tf.keras.layers.GRU(rnn_units,
                        dropout = 0.5,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),                       
  tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))

print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'h God is just,\nHe be as miserably slain as I.\nAh, let me live in prison all my days;\nAnd when I give'
Next Char Predictions: 
 ".wsNBDK$&mEA,ujXh&RdDKwiNC!\nfNSdI',;ecy&uFyewJMtmBK$V\n3PYmpzhSQoq$HPt-mQNoZmKTXmXIdD?UO?AuK3Z lNjWAY"


In [25]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1754622


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [29]:
#@title Default title text
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
172/172 [==============================] - 35s 202ms/step - loss: 2.7190
Epoch 2/10
172/172 [==============================] - 33s 195ms/step - loss: 2.0413
Epoch 3/10
172/172 [==============================] - 33s 193ms/step - loss: 1.7908
Epoch 4/10
172/172 [==============================] - 32s 188ms/step - loss: 1.6357
Epoch 5/10
172/172 [==============================] - 32s 186ms/step - loss: 1.5356
Epoch 6/10
172/172 [==============================] - 33s 190ms/step - loss: 1.4656
Epoch 7/10
172/172 [==============================] - 34s 195ms/step - loss: 1.4119
Epoch 8/10
172/172 [==============================] - 34s 196ms/step - loss: 1.3678
Epoch 9/10
172/172 [==============================] - 33s 194ms/step - loss: 1.3297
Epoch 10/10
172/172 [==============================] - 33s 192ms/step - loss: 1.2940


In [0]:
EPOCHS=15

In [33]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/15
172/172 [==============================] - 32s 188ms/step - loss: 1.2491
Epoch 2/15
172/172 [==============================] - 33s 195ms/step - loss: 1.2185
Epoch 3/15
172/172 [==============================] - 34s 199ms/step - loss: 1.1875
Epoch 4/15
172/172 [==============================] - 33s 192ms/step - loss: 1.1576
Epoch 5/15
172/172 [==============================] - 32s 186ms/step - loss: 1.1277
Epoch 6/15
172/172 [==============================] - 32s 187ms/step - loss: 1.1011
Epoch 7/15
172/172 [==============================] - 32s 183ms/step - loss: 1.0772
Epoch 8/15
172/172 [==============================] - 32s 185ms/step - loss: 1.0578
Epoch 9/15
172/172 [==============================] - 32s 184ms/step - loss: 1.0409
Epoch 10/15
172/172 [==============================] - 32s 185ms/step - loss: 1.0259
Epoch 11/15
172/172 [==============================] - 32s 185ms/step - loss: 1.0151
Epoch 12/15
172/172 [==============================] - 32s 183ms/step - lo

In [34]:
tf.train.latest_checkpoint(checkpoint_dir)

SyntaxError: ignored

In [0]:
model = build_model(  vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.

def generate_text(model, start_string,temperature):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))



In [37]:
print(generate_text(model, start_string=u"ROMEO: ",temperature = 1))

ROMEO: Camillo!

MARTIUS:
Ond man, sir, this lievy dather wrong,
But manf another tender mercos?
Be not ablet. All the wholesomeoble heat.
Good Kate, I pray thee, Bianca bear my
sweet Exeter, hear me been,
Or fair As interrale of the king's ship.
These brings, man i'dtime, that we may so near?

First Senator:
Speak upon me, yes, you love me, blesseepo you, behels more powdecs you
have not a maid;
The very pingom of the wild-gaze ignot this extreme cheers,
Of this pollow-crown and to my good armitral:
So honour is a villany.
What, ig mine honour, his rume death.

MENENIUS:
Well, we were stoped in our pear,
Have I promised, and to redeem me fares
From the devil madest burn his present,,
But then you louse. For some conjuct his this,
Benrown alike and now repent the engremlly
Tread on the woe! the king cannot accuse
Of contepted as a tailors false,--
Let them as much milding at your disposition
That which you rugut was unteronate rudely; sours all abroved
At our revort's like a valuarve, 

In [38]:
## changing the temperature to 0.7
print(generate_text(model, start_string=u"ROMEO: ",temperature = 0.7))

ROMEO: he's a head! down, knee--
Who each one heaven should be aveited
But that the villain's tamous and done,
When he doth courtempore to me, and only my tongue for their
tongue!

Second Citizen:
And with that regies, he will return brave thee,
When et stand heart to have some cause!

CLIFFORD:
I would they say,
to strike agone.

MIRANDA:
I will reselves with thee; here in this valour,
That he did laugh all maving marriage fet since I am sound,
But not a word, that he waters, sword,
Which so hoth peeving rispositions of yourselves come
As thou shouldst pass hereafter than the earth,
That now in cord again before.

CORIOLANUS:
What must I will do't. Indeed, it is a great insontate,
Who will beguine me, but as a very day.

BIONDELLO:
Why, here hear me speak of death.

CLARENCE:
And yet perform'd a forward kindness, my prayers do
the topt with him.

Provost:
What is a very day, that I may say amen
To see the Duke of Buckingham, as the best of their grave.

LUCENTIO:
A half, sir.

AUTOLYC

In [39]:
## changing the temperature to 0.7
print(generate_text(model, start_string=u"ROMEO: ", temperature = 0.5 ))

ROMEO: thou shalt be a victory:
My father, and my love to hear of heaven,
To fain desert of any thing about me.

Provost:
A very surely, sir, the voice of fair deadest store.

PROSPERO:
Soft and all this fool and not thy way.

KING RICHARD II:
What is the matter?

ANGELO:
What is the news i' the company fires after?

CAMILLO:
Sir, I would they would have in your comfort night.

RICHARD:
Saddest lamentabous man, then was too soon,
I'll to my way of compassion and revenge
To mourt at as have been my stays,
When banished earth, we should not be a cause
To pray the gates of the house of York
Through to the angient world,
To lose his repent.

LEONTES:
Thou shalt not give me thy son:
This squared soul, thou think should have had some confidence.

KING EDWARD IV:
Thank me, I'll tell thee: then I would proceed
With conscience and my love, and look'd his repent,
My inconstant will ever to the ground,
And to the world I would have lazed for doom:
I have a trinking maid I see thee hence,
The love

In [0]:
  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.

def generate_text_wtfs(model, start_string,temperature):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 100

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string if s in char2idx]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return ( ''.join(text_generated))



In [41]:
!pip install rouge

In [0]:
from rouge import Rouge 
rouge = Rouge()

In [56]:
## Just took the sequences of the first few sentences
# and tried to compare predicted sentences with next sentences
all_str_sequences = []

for i,item in enumerate(sequences.take(5)):
  all_str_sequences.append(repr(''.join(idx2char[item.numpy()])))


for i,item in enumerate(all_str_sequences):
  predicted_text = generate_text_wtfs(model, start_string= item,temperature = 1)
  print(item)
  if i + 1 < len(all_str_sequences):
    scores = rouge.get_scores(predicted_text, all_str_sequences[i+1])
    print(scores)

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
[{'rouge-1': {'f': 0.06896551239001224, 'p': 0.058823529411764705, 'r': 0.08333333333333333}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.06520102394148532, 'p': 0.058823529411764705, 'r': 0.08333333333333333}}]
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
[{'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}]
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
[{'rouge-1': {'f': 0.1052631529085875, 'p': 0.1111111111111111, 'r': 0.1}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.10468478889492801, 'p': 0.1111111111111111, 'r': 0.1}}]
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: 

# Here as we increase the number of epochs model learns more about this huge data hence the loss decreases